TESTING GEOGRAPHICALLY WEIGHTED REGRESSION . Fotheringham, Brunsdon & Charlton (2003)

In [2]:
import pandas as pd
import numpy as np
file_path= "/content/austinHousingData.csv"
data = pd.read_csv(file_path)

In [3]:
#USELESS FEATURES ## BUT WE DROP "city" after using it. ## no dropping yet
useless_features= ['zipcode','zpid', 'city', 'streetAddress', 'zipcode', 'description','homeImage','homeType', 'latestPriceSource']

In [ ]:
###################### NEW VARIABLES #################

In [22]:
data['zipcode'].nunique()

48

In [23]:
zp = pd.read_csv("/content/uszips.csv")
zp.tail()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
33778,99923,55.98043,-130.03803,Hyder,AK,Alaska,True,NaN,25.0,0.6,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Sitka
33779,99925,55.55398,-132.96276,Klawock,AK,Alaska,True,NaN,854.0,6.1,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Sitka
33780,99926,55.12617,-131.48928,Metlakatla,AK,Alaska,True,NaN,1385.0,4.0,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Metlakatla
33781,99927,56.33305,-133.60044,Point Baker,AK,Alaska,True,NaN,18.0,1.5,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Sitka
33782,99929,56.36089,-132.00635,Wrangell,AK,Alaska,True,NaN,2105.0,0.3,2275,Wrangell,"{""02275"": 100}",Wrangell,02275,False,False,America/Sitka


In [4]:
data['latest_saledate'].dtypes

dtype('O')

In [5]:
data['latest_saledate']

,latest_saledate
0,2019-09-02
1,2020-10-13
2,2019-07-31
3,2018-08-08
4,2018-10-31
...,...
15166,2020-12-10
15167,2020-12-16
15168,2021-01-27
15169,2019-08-07


In [6]:
data['latest_saledate'] = data['latest_saledate'].str.strip()
data['latest_saledate'] = pd.to_datetime(data['latest_saledate'], errors='coerce')
print(data['latest_saledate'].isna().sum())

0


In [7]:
# Data inspection
print(f"integer:{data.select_dtypes(include='integer').columns}")
print(f"object:{data.select_dtypes(include='object').columns}")
print(f"bools:{data.select_dtypes(include='bool').columns}")

integer:Index(['zpid', 'zipcode', 'garageSpaces', 'parkingSpaces', 'yearBuilt',
       'numPriceChanges', 'latest_salemonth', 'latest_saleyear', 'numOfPhotos',
       'numOfAccessibilityFeatures', 'numOfAppliances', 'numOfParkingFeatures',
       'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures',
       'numOfWaterfrontFeatures', 'numOfWindowFeatures',
       'numOfCommunityFeatures', 'numOfPrimarySchools',
       'numOfElementarySchools', 'numOfMiddleSchools', 'numOfHighSchools',
       'avgSchoolSize', 'MedianStudentsPerTeacher', 'numOfBedrooms',
       'numOfStories'],
      dtype='object')
object:Index(['city', 'streetAddress', 'description', 'homeType', 'latestPriceSource',
       'homeImage'],
      dtype='object')
bools:Index(['hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView'],
      dtype='object')


In [8]:
 # We have 958 days of analysis
 data['latest_saledate'].value_counts().sort_index()

,count
latest_saledate,
2018-01-22,4
2018-01-23,10
2018-01-24,8
2018-01-25,11
2018-01-26,13
...,...
2021-01-26,2
2021-01-27,2
2021-01-28,1


In [9]:
#################### 1 CITY ONLY ###############################
df= data[data['city'] == 'austin'].sort_values(by='latest_saledate')
df.select_dtypes(include='integer').columns

Index(['zpid', 'zipcode', 'garageSpaces', 'parkingSpaces', 'yearBuilt',
       'numPriceChanges', 'latest_salemonth', 'latest_saleyear', 'numOfPhotos',
       'numOfAccessibilityFeatures', 'numOfAppliances', 'numOfParkingFeatures',
       'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures',
       'numOfWaterfrontFeatures', 'numOfWindowFeatures',
       'numOfCommunityFeatures', 'numOfPrimarySchools',
       'numOfElementarySchools', 'numOfMiddleSchools', 'numOfHighSchools',
       'avgSchoolSize', 'MedianStudentsPerTeacher', 'numOfBedrooms',
       'numOfStories'],
      dtype='object')

In [10]:
data['latest_saledate'].dropna()

,latest_saledate
0,2019-09-02
1,2020-10-13
2,2019-07-31
3,2018-08-08
4,2018-10-31
...,...
15166,2020-12-10
15167,2020-12-16
15168,2021-01-27
15169,2019-08-07


In [11]:
object_features = ['city', 'streetAddress', 'description', 'homeType', 'latestPriceSource','homeImage']

In [12]:
df.select_dtypes(include='object').columns

Index(['city', 'streetAddress', 'description', 'homeType', 'latestPriceSource',
       'homeImage'],
      dtype='object')

In [13]:
################# FINAL DATAFRAME IN USE ################
df_1 = df.drop(columns=useless_features, errors='ignore')
df_1.shape

(15020, 39)

In [14]:
# FINAL CHECK BEFORE MERGING
df_1.isna().sum()

,0
latitude,0
longitude,0
propertyTaxRate,0
garageSpaces,0
hasAssociation,0
hasCooling,0
hasGarage,0
hasHeating,0
hasSpa,0
hasView,0


In [15]:
# Get Mortgage rate
import pandas as pd
MTG = pd.read_csv('/content/MORTGAGE30US.csv')

print(MTG.shape)

MPPSQ = pd.read_csv('/content/MEDLISPRIPERSQUFEE12420.csv')
print(MPPSQ.dtypes)

(160, 2)
observation_date           object
MEDLISPRIPERSQUFEE12420     int64
dtype: object


In [16]:
############################################################################### Tranforming Mortgage Data #############################################################################################

In [17]:
################## TREATING DATES ON EXTERNAL MACRO VARIABLES ##################
def parse_obs_date(date_str):
    d = str(date_str).strip().split()[0]
    if "/" in d:
        return pd.to_datetime(d, format='%d/%m/%Y', errors='raise')
    else:
        return pd.to_datetime(d, errors='raise')

MTG['observation_date'] = MTG['observation_date'].apply(parse_obs_date)
MPPSQ['observation_date'] = MPPSQ['observation_date'].apply(parse_obs_date)
MTG['MORTGAGE30US'].dtypes

dtype('float64')

In [18]:
print(MTG.isna().sum())
print(MPPSQ.isna().sum())

observation_date    0
MORTGAGE30US        0
dtype: int64
observation_date           0
MEDLISPRIPERSQUFEE12420    0
dtype: int64


In [19]:
df_2 = df_1.copy()

In [20]:
##################################.MERGE. ####################################

import pandas as pd

df_2['latest_saledate'] = pd.to_datetime(df_2['latest_saledate'], errors='coerce')
df_2 = df_2.sort_values('latest_saledate')

# Read and prepare the MTG rates
MTG['observation_date'] = pd.to_datetime(MTG['observation_date'], format='%d/%m/%Y', errors='coerce')
MTG = MTG.sort_values('observation_date')

# Merge using asof so that each sale date gets the most recent MTG rate available
df_2 = pd.merge_asof(df_2, MTG, left_on='latest_saledate', right_on='observation_date', direction='backward')

# Optional: Remove the duplicate 'date' column if no longer need it
df_2.drop(columns=['observation_date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories', 'MORTGAGE30US'],
      dtype='object')

In [21]:
# read variable, sort variable on date
MPPSQ['observation_date'] = pd.to_datetime(MPPSQ['observation_date'], format='%d/%m/%Y', errors='coerce')
MPPSQ = MPPSQ.sort_values('observation_date')

# Merge using asof so that each sale date gets the most recent MPPSQ available
df_2 = pd.merge_asof(df_2, MPPSQ, left_on='latest_saledate', right_on='observation_date', direction='backward')

# Optional: Remove the duplicate 'date' column if you no longer need it
df_2.drop(columns=['observation_date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'numOfPhotos', 'numOfAccessibilityFeatures',
       'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfPrimarySchools', 'numOfElementarySchools',
       'numOfMiddleSchools', 'numOfHighSchools', 'avgSchoolDistance',
       'avgSchoolRating', 'avgSchoolSize', 'MedianStudentsPerTeacher',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories', 'MORTGAGE30US',
       'MEDLISPRIPERSQUFEE12420'],
      dtype='object')

In [ ]:
df_2[['latest_saledate','MORTGAGE30US','MEDLISPRIPERSQUFEE12420']]

,latest_saledate,MORTGAGE30US,MEDLISPRIPERSQUFEE12420
0,2018-01-22,4.04,163
1,2018-01-22,4.04,163
2,2018-01-22,4.04,163
3,2018-01-22,4.04,163
4,2018-01-23,4.04,163
...,...,...,...
15015,2021-01-27,2.77,226
15016,2021-01-28,2.73,226
15017,2021-01-29,2.73,226
15018,2021-01-29,2.73,226


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
df_2.to_pickle("/content/drive/MyDrive/lions_den_data/df_2.pkl")

In [ ]:
## run -> on next script -> df_2 = pd.read_pickle('/content/drive/MyDrive/lions_den_data/df_2.pkl')
